In [1]:
# !pip install -q -U langdetect --no-index --find-links /kaggle/input/pip-install-langdetect

In [2]:
!ipython /kaggle/input/gemma2-inference-0805/inference-gemma-2-9b-0805.ipynb

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2649: UserWarning: You executed the system command !pip which may not work as expected. Try the IPython magic %pip instead.
  warnings.warn(
Looking in links: /kaggle/input/lmsys-wheel-files/lmsys
Processing /kaggle/input/lmsys-wheel-files/lmsys/peft-0.11.1-py3-none-any.whl
Processing /kaggle/input/lmsys-wheel-files/lmsys/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl
2024-08-05 05:45:28.724814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 05:45:28.724956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 05:45:28.876035: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory:

In [3]:
import pandas as pd
submission_1 = pd.read_csv('/kaggle/working/submission.csv')

In [4]:
submission_1

,id,winner_model_a,winner_model_b,winner_tie
0,1233961,0.103607,0.692299,0.204094
1,136060,0.002074,0.968699,0.029227
2,211333,0.239302,0.410234,0.350464


In [5]:
!ipython /kaggle/input/llama3-1-8b-inference-notebook/Llama3.1-base2-inference.ipynb

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2649: UserWarning: You executed the system command !pip which may not work as expected. Try the IPython magic %pip instead.
  warnings.warn(
2024-08-05 05:49:25.977732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 05:49:25.977790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 05:49:25.979267: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
]0;IPython: kaggle/workingMap:   0%|          | 0/3 [00:00<?, ? examples/s]
CPU times: user 543 ms, sys: 79.1 ms, total: 622 ms
Wall time: 770 ms
<|b

In [6]:
submission_2 = pd.read_csv('/kaggle/working/submission.csv')

# 損失値
loss_1 = 0.909  # submission_1 の損失
loss_2 = 0.919  # submission_2 の損失

# 重みの計算
weight_1 = 1 / loss_1
weight_2 = 1 / loss_2

# 重みの正規化
total_weight = weight_1 + weight_2
norm_weight_1 = weight_1 / total_weight
norm_weight_2 = weight_2 / total_weight

print('norm_1', norm_weight_1)
print('norm_2', norm_weight_2)


submission = pd.DataFrame()
data = submission_2.merge(submission_1, on='id', how='left')
submission['id'] = data['id']
# 加重平均の計算
submission['winner_model_a'] = (data['winner_model_a_x'] * norm_weight_2 + data['winner_model_a_y'] * norm_weight_1)
submission['winner_model_b'] = (data['winner_model_b_x'] * norm_weight_2 + data['winner_model_b_y'] * norm_weight_1)
submission['winner_tie'] = (data['winner_tie_x'] * norm_weight_2 + data['winner_tie_y'] * norm_weight_1)
# submission.to_csv('submission.csv', index=False)
# display(submission)


norm_1 0.5027352297592999
norm_2 0.4972647702407002


In [7]:
# import pandas as pd
# from langdetect import detect
# from tqdm import tqdm
# import time

# test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
# # 言語を検出し、ラベルを追加する関数
# def label_language(text):
#     try:
#         lang = detect(text)
#         if lang == 'en':
#             return 'English'
#         else:
#             return 'Other'
#     except:
#         return 'Unknown'


In [8]:
# %%time
# test['language'] = test['prompt'].apply(label_language)

# other_language_ids = test[test['language'] == 'Other']['id'].tolist()

# # submission データフレームを効率的に更新
# mask = submission['id'].isin(other_language_ids)
# columns_to_update = ['winner_model_a', 'winner_model_b', 'winner_tie']

# for test_id in other_language_ids:
#     #英語以外の言語が格納されている場合、Gemma2の予測のみを利用する
#     submission.loc[submission['id'] == test_id, columns_to_update] = submission_1[submission_1['id']==test_id][columns_to_update]

In [9]:
submission.to_csv('submission.csv', index=False)
display(submission)

,id,winner_model_a,winner_model_b,winner_tie
0,1233961,0.104938,0.674972,0.220090
1,136060,0.009114,0.939879,0.051007
2,211333,0.299506,0.345801,0.354693
